In [1]:
# importing packages
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import duckdb

# __Data Collection and Cleaning:__

The data that we collected was from multiple different dataframes (more information below in the data description). We made sure to find data that matched in terms of scope (timeframe, location, etc.), but the data was structured in different ways. So, we clean up and merge all of the data below.

First, we will read in all of the data that we collected regarding the sale of houses from Jan 2018-Sep 2023. This data was taken from https://www.zillow.com/research/data/, and we downloaded 4 data tables from here.

In [2]:
df_new_listings = pd.read_csv("new-listings.csv")
df_median_days_to_pending = pd.read_csv("median_days_to_pending_month.csv")
df_mean_sale_to_list = pd.read_csv("mean_sale_to_list_month.csv")
df_pct_sold_below_list = pd.read_csv("pct_sold_below_list_month.csv")

First, we will work with the `df_new_listings` dataframe. The data consists of the number of listings that were posted each month for various regions around the United States. Our project is only concerned with the general US–this is the data found in the first row of the data set. Since we’re examining information on the whole nation and not specific areas, the columns that describe the locations are no longer necessary. We will remove them by using SQL. We print a list of the column names before, and the dataframe after to ensure that the process was done correctly.

In [3]:
print("Original column names:")
print(df_new_listings.columns)
df_new_listings = duckdb.sql("SELECT * EXCLUDE ('RegionID', \
    'SizeRank', 'RegionName', 'RegionType', 'StateName') \
    FROM df_new_listings").df()
print("\nDataframe after removing unneeded columns:")
print(df_new_listings.head())

Original column names:
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31',
       '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31',
       '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31',
       '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
       '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
       '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
       '2020-09-30', '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31',
       '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
       '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
       '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
       '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
       '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-

Now, we want to reshape our data (which is currently only 1 single row) to be vertical, as this will be more useful once we try to merge the rest of the data together. Since our data is currently a series (just the first row of the original table), we will do this by making a new dataframe using the series. Also for the sake of merging, we’ll create `Year` and `Month` columns by turning `Date` to a datetime and then extracting the month and year. We’ll print the first five rows of the new dataframe.

In [4]:
df_new_listings = df_new_listings.iloc[0]
df_new_listings = pd.DataFrame({'Date': df_new_listings.index, \
                                      'New_Listings': df_new_listings})
df_new_listings['Date'] = pd.to_datetime(df_new_listings['Date'])
df_new_listings['Year'] = df_new_listings['Date'].dt.year
df_new_listings['Month'] = df_new_listings['Date'].dt.month


print(df_new_listings.head())

                 Date  New_Listings  Year  Month
2018-03-31 2018-03-31     1421529.0  2018      3
2018-04-30 2018-04-30     1500194.0  2018      4
2018-05-31 2018-05-31     1592414.0  2018      5
2018-06-30 2018-06-30     1660614.0  2018      6
2018-07-31 2018-07-31     1709142.0  2018      7


We will now do the same thing for `df_median_days_to_pending`. All of the data that we got from Zillow came in the same format, so following the same steps as above will allow us to make our reshaped data consistent. So, below we take the first row, representing data for the whole US, then remove unneeded columns.

In [5]:
df_median_days_to_pending = duckdb.sql("SELECT * EXCLUDE ('RegionID', \
    'SizeRank', 'RegionName', 'RegionType', 'StateName') \
    FROM df_median_days_to_pending").df()

Now we turn the data into a new data frame:

In [6]:
average_median_days = df_median_days_to_pending.iloc[0]
df_median_days_to_pending = pd.DataFrame({'Date': \
                                       average_median_days.index, \
                                       'Days_To_Pending': \
                                          average_median_days})

Now, add columns that we will use to merge, and print the first five rows.

In [7]:
df_median_days_to_pending['Date'] = \
pd.to_datetime(df_median_days_to_pending['Date'])

df_median_days_to_pending['Year'] = \
df_median_days_to_pending['Date'].dt.year

df_median_days_to_pending['Month'] = \
df_median_days_to_pending['Date'].dt.month

print(df_median_days_to_pending.head())

                 Date  Days_To_Pending  Year  Month
2018-01-31 2018-01-31             47.0  2018      1
2018-02-28 2018-02-28             25.0  2018      2
2018-03-31 2018-03-31             20.0  2018      3
2018-04-30 2018-04-30             18.0  2018      4
2018-05-31 2018-05-31             18.0  2018      5


`df_pct_sold_below_list` will follow the same procedure, and we will print the first five rows.. 

In [8]:
df_pct_sold_below_list = duckdb.sql("SELECT * EXCLUDE ('RegionID', \
    'SizeRank', 'RegionName', 'RegionType', 'StateName') \
    FROM df_pct_sold_below_list").df()

In [9]:
average_sold_below = df_pct_sold_below_list.iloc[0]
df_pct_sold_below_list = pd.DataFrame({'Date': \
                                       average_sold_below.index,\
                            'Percent_Sold_Below_List': \
                                       average_sold_below})
df_pct_sold_below_list['Date'] = \
    pd.to_datetime(df_pct_sold_below_list['Date'])
df_pct_sold_below_list['Year'] = df_pct_sold_below_list['Date'].dt.year
df_pct_sold_below_list['Month'] = df_pct_sold_below_list['Date'].dt.month
print(df_pct_sold_below_list.head())

                 Date  Percent_Sold_Below_List  Year  Month
2018-01-31 2018-01-31                 0.642516  2018      1
2018-02-28 2018-02-28                 0.619052  2018      2
2018-03-31 2018-03-31                 0.577498  2018      3
2018-04-30 2018-04-30                 0.551780  2018      4
2018-05-31 2018-05-31                 0.533494  2018      5


Now, again, `df_mean_sale_to_list` will be cleaned using the same procedure. The first five rows are printed.

In [10]:
df_mean_sale_to_list = duckdb.sql("SELECT * EXCLUDE ('RegionID', \
    'SizeRank', 'RegionName', 'RegionType', \
    'StateName') FROM df_mean_sale_to_list").df()

In [11]:
df_mean_sale_to_list = df_mean_sale_to_list.iloc[0]
df_mean_sale_to_list = pd.DataFrame({'Date': \
                                       df_mean_sale_to_list.index, \
                                       'Sale_To_List_Price_Ratio': \
                                       df_mean_sale_to_list})
df_mean_sale_to_list['Date'] = \
pd.to_datetime(df_mean_sale_to_list['Date'])
df_mean_sale_to_list['Year'] = df_mean_sale_to_list['Date'].dt.year
df_mean_sale_to_list['Month'] = df_mean_sale_to_list['Date'].dt.month
print(df_mean_sale_to_list.head())

                 Date  Sale_To_List_Price_Ratio  Year  Month
2018-01-31 2018-01-31                  0.976664  2018      1
2018-02-28 2018-02-28                  0.978713  2018      2
2018-03-31 2018-03-31                  0.983239  2018      3
2018-04-30 2018-04-30                  0.986186  2018      4
2018-05-31 2018-05-31                  0.987151  2018      5


Next, we read in the data that we found on birth and mortality rates, which was taken from https://wonder.cdc.gov/natality-expanded-current.html and https://wonder.cdc.gov/ucd-icd10-expanded.html, respectively. As this data was retrieved by submitting a request, we were unable to scrape it, so we took the data, and turned it into a Excel sheet. Some manual cleaning then had to occur: there were rows after each year that totaled up the numbers for the year, so those were deleted, and the "Year" column was expanded to fill in all of the rows (originally only filled out for January of each year). Then the data was converted to a csv. In the dataset, numbers in the thousands contain commas, so we specify that to turn them into ints.

In [12]:
birth_df = pd.read_csv("birth_rate.csv", thousands = ",")
print(birth_df.head())

   Year     Month  Births
0  2018   January  314808
1  2018  February  284250
2  2018     March  316044
3  2018     April  298394
4  2018       May  320622


When looking at these later datasets that we found, we noticed that they all had their months formatted differently. This would make merging difficult, so for these we made sure that the months were all in "mmm" format. We do it for `birth_df` here:

In [13]:
birth_df["Month"] = birth_df["Month"].apply(lambda x: x[:3].lower())
print(birth_df.head())

   Year Month  Births
0  2018   jan  314808
1  2018   feb  284250
2  2018   mar  316044
3  2018   apr  298394
4  2018   may  320622


Now we will read in the mortality rate dataset, and do the same thing that we did with `birth_df`: address the commas, and put the months into the format that we wanted.

In [14]:
mortality_df = pd.read_csv("mortality_rate.csv", thousands = ",")
mortality_df["Month"] = \
    mortality_df["Month"].apply(lambda x: x[:3].lower())
print(mortality_df.head())

   Year Month  Deaths
0  2018   jan  286744
1  2018   feb  236998
2  2018   mar  248805
3  2018   apr  233164
4  2018   may  228772


Now, we will work with the data on inflation and unemployment rates. The inflation data came from https://www.usinflationcalculator.com/inflation/current-inflation-rates/ and the unemployment data came from https://data.bls.gov/timeseries/LNS14000000. We got the data as xlsx files, and the converted to csv. Both contained data that was before 2018, so we took only the rows that were for 2018 or later.

These datasets are going to require more cleaning than `birth_df` and `mortality_df`, so we will print out what they look like.

In [15]:
inflation_df = pd.read_csv("inflation_rate.csv")

unemployment_df = pd.read_csv("unemployment_rate.csv")
unemployment_df = unemployment_df.loc[unemployment_df['Year'] >= 2018]

print(inflation_df)
print(unemployment_df)

   Year  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
0  2023  6.4  6.0  5.0  4.9  4.0  3.0  3.2  3.7  3.7  NaN  NaN  NaN
1  2022  7.5  7.9  8.5  8.3  8.6  9.1  8.5  8.3  8.2  7.7  7.1  6.5
2  2021  1.4  1.7  2.6  4.2  5.0  5.4  5.4  5.3  5.4  6.2  6.8  7.0
3  2020  2.5  2.3  1.5  0.3  0.1  0.6  1.0  1.3  1.4  1.2  1.2  1.4
4  2019  1.6  1.5  1.9  2.0  1.8  1.6  1.8  1.7  1.7  1.8  2.1  2.3
5  2018  2.1  2.2  2.4  2.5  2.8  2.9  2.9  2.7  2.3  2.5  2.2  1.9
    Year  Jan  Feb  Mar   Apr   May   Jun   Jul  Aug  Sep  Oct  Nov  Dec
5   2018  4.0  4.1  4.0   4.0   3.8   4.0   3.8  3.8  3.7  3.8  3.8  3.9
6   2019  4.0  3.8  3.8   3.6   3.7   3.6   3.7  3.7  3.5  3.6  3.6  3.6
7   2020  3.5  3.5  4.4  14.7  13.2  11.0  10.2  8.4  7.9  6.9  6.7  6.7
8   2021  6.3  6.2  6.1   6.1   5.8   5.9   5.4  5.2  4.8  4.5  4.2  3.9
9   2022  4.0  3.8  3.6   3.6   3.6   3.6   3.5  3.7  3.5  3.7  3.6  3.5
10  2023  3.4  3.6  3.5   3.4   3.7   3.6   3.5  3.8  3.8  NaN  NaN  NaN


Currently, they are wide dataframes. To be consistent with all of the other data, we need to have the month and years as columns, so both of these dataframes need reshaping. We will also standardize the representation of the months.

In [16]:
inflation_df = inflation_df.melt(id_vars="Year", \
                                 var_name="Month", \
                                 value_name ="Inflation_Rate")
inflation_df["Month"] = \
    inflation_df["Month"].apply(lambda x: x[:3].lower())
print(inflation_df.head())

   Year Month  Inflation_Rate
0  2023   jan             6.4
1  2022   jan             7.5
2  2021   jan             1.4
3  2020   jan             2.5
4  2019   jan             1.6


In [17]:
unemployment_df = unemployment_df.melt(id_vars="Year", \
                                       var_name="Month", \
                                       value_name ="Unemployment_Rate")
unemployment_df["Month"] = \
                unemployment_df["Month"].apply(lambda x: x[:3].lower())
print(unemployment_df.head())

   Year Month  Unemployment_Rate
0  2018   jan                4.0
1  2019   jan                4.0
2  2020   jan                3.5
3  2021   jan                6.3
4  2022   jan                4.0


__Merging Dataframes__

Now that we have gotten our data to be in the same structure, we can merge them together into one larger data set. We will start by merging all of our data on the sale of houses into one dataframe that we will call `housing_data_df`. We print the head to get a preview of what it looks like.

In [18]:
housing_data_df = duckdb.sql("SELECT a.Year, a.Month, \
New_Listings, Days_To_Pending, \
Percent_Sold_Below_List, Sale_To_List_Price_Ratio \
FROM df_median_days_to_pending AS a \
LEFT JOIN df_new_listings AS b \
    ON a.Year = b.Year AND a.Month = b.Month \
LEFT JOIN df_pct_sold_below_list AS c \
    ON a.Year = c.Year AND a.Month = c.Month \
LEFT JOIN df_mean_sale_to_list AS d \
    ON a.Year = d.Year AND a.Month = d.Month \
ORDER BY a.Year, a.Month").df()

Now, the data here is all merged, and in order. However, we had these dataframes represent months using integers, so we need to change this column so it matches our other data.

https://stackoverflow.com/questions/14533709/basic-python-programming-to-convert-month-number-to-month-name-using-dictionary

Using the function provided in stackoverflow, we converted the column of month integers to strings to match the data type in the other dataframes. We used the pandas apply function with a lambda function to apply the dictionary to each value in the column.


In [19]:
monthDict={1:'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', \
           6:'jun', 7:'jul', 8:'aug', 9:'sep', 10:'oct', \
           11:'nov', 12:'dec'}

In [20]:
housing_data_df["Month"] = \
    housing_data_df["Month"].apply(lambda x: monthDict[x])

print (housing_data_df.head())

   Year Month  New_Listings  Days_To_Pending  Percent_Sold_Below_List  \
0  2018   jan           NaN             47.0                 0.642516   
1  2018   feb           NaN             25.0                 0.619052   
2  2018   mar     1421529.0             20.0                 0.577498   
3  2018   apr     1500194.0             18.0                 0.551780   
4  2018   may     1592414.0             18.0                 0.533494   

   Sale_To_List_Price_Ratio  
0                  0.976664  
1                  0.978713  
2                  0.983239  
3                  0.986186  
4                  0.987151  


Now, we can move on to merging in the other four data tables.

In [21]:
housing_data_df = duckdb.sql("SELECT a.*, Births, Deaths, Inflation_rate, \
Unemployment_Rate FROM housing_data_df AS a \
LEFT JOIN mortality_df AS b ON a.Year = b.Year AND a.Month = b.Month \
LEFT JOIN inflation_df AS c ON a.Year = c.Year AND a.Month = c.Month \
LEFT JOIN unemployment_df AS d ON a.Year = d.Year AND a.Month = d.Month \
LEFT JOIN birth_df AS e on a.Year = e.Year AND a.Month = e.Month \
ORDER BY a.YEAR").df()
housing_data_df

,Year,Month,New_Listings,Days_To_Pending,Percent_Sold_Below_List,Sale_To_List_Price_Ratio,Births,Deaths,Inflation_Rate,Unemployment_Rate
0,2018,jan,NaN,47.0,0.642516,0.976664,314808.0,286744.0,2.1,4.0
1,2018,feb,NaN,25.0,0.619052,0.978713,284250.0,236998.0,2.2,4.1
2,2018,mar,1421529.0,20.0,0.577498,0.983239,316044.0,248805.0,2.4,4.0
3,2018,apr,1500194.0,18.0,0.551780,0.986186,298394.0,233164.0,2.5,4.0
4,2018,may,1592414.0,18.0,0.533494,0.987151,320622.0,228772.0,2.8,3.8
...,...,...,...,...,...,...,...,...,...,...
64,2023,may,880510.0,10.0,0.419565,0.997535,NaN,252046.0,4.0,3.7
65,2023,jun,907228.0,11.0,0.399708,1.000553,NaN,240140.0,3.0,3.6
66,2023,jul,930911.0,12.0,0.411186,0.998900,NaN,244278.0,3.2,3.5
67,2023,aug,950306.0,13.0,0.434983,0.995926,NaN,238615.0,3.7,3.8


We'll make a few final touches to our dataset. First of all, it was mentioned to us that these events probably do not have an immediate impact on the economy, and therefore will not have an immediate impact on the economy, and therefore won't have an immediate impact on the sale of houses. We will adjust the `Deaths` column by 3 months to account for time between when an estate is resold [1]. It would be interesting to do more analyses with different adjustments of time to determine how long it actually takes for impacts to be seen.

We will also create a column that measures the amount of months since the starting date of our data, January 2018. This will be useful in our later analyses where we need to account for time.

[1] https://www.homelight.com/blog/how-long-does-executor-have-to-sell-a-house/ 

In [22]:
housing_data_df['Deaths'] = pd.concat([pd.Series([np.NaN]*3), housing_data_df['Deaths'][:-3]], ignore_index=True)
housing_data_df['Months_Since_Jan_2018'] = housing_data_df.index
housing_data_df

,Year,Month,New_Listings,Days_To_Pending,Percent_Sold_Below_List,Sale_To_List_Price_Ratio,Births,Deaths,Inflation_Rate,Unemployment_Rate,Months_Since_Jan_2018
0,2018,jan,NaN,47.0,0.642516,0.976664,314808.0,NaN,2.1,4.0,0
1,2018,feb,NaN,25.0,0.619052,0.978713,284250.0,NaN,2.2,4.1,1
2,2018,mar,1421529.0,20.0,0.577498,0.983239,316044.0,NaN,2.4,4.0,2
3,2018,apr,1500194.0,18.0,0.551780,0.986186,298394.0,286744.0,2.5,4.0,3
4,2018,may,1592414.0,18.0,0.533494,0.987151,320622.0,236998.0,2.8,3.8,4
...,...,...,...,...,...,...,...,...,...,...,...
64,2023,may,880510.0,10.0,0.419565,0.997535,NaN,249084.0,4.0,3.7,64
65,2023,jun,907228.0,11.0,0.399708,1.000553,NaN,269173.0,3.0,3.6,65
66,2023,jul,930911.0,12.0,0.411186,0.998900,NaN,251959.0,3.2,3.5,66
67,2023,aug,950306.0,13.0,0.434983,0.995926,NaN,252046.0,3.7,3.8,67


Now, the data is all in one table and looks the way we would like it to. There are still empty cells with missing data, so we will address those when we analyze the data.

Now we will convert the dataframe into a csv file.

In [23]:
housing_data_df.to_csv('housing_data_df.csv', index=False)